In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import os
from scipy.io import wavfile
import re
import seaborn as sns
import random
from sklearn import svm
import librosa
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import re
import tensorflow as tf
from tensorflow.keras.utils import to_categorical
from keras.utils.np_utils import to_categorical


In [5]:
# test data and test data directory 

data_dir = "/content/drive/MyDrive/ELEC378_FinalProject/data/data"

test_dir = "/content/drive/MyDrive/ELEC378_FinalProject/test/test"



Containis function for feature extraction. Takes a directory and a boolean indicating wheather to split the data or not. For cross validation use the data directory (labeled data) and split = true. Run all the way to the cell that prints out accuracy score. For testing model on the unlabled data in test folder (those that are named sample001, etc), run the cells below the accuracy score cell will give you a .csv file ready for submission on kaggle. 

In [6]:
#@title Feature Extraction func
def FeatureExtraction (dir, split=True): 
 
  '''
  Function for feature extraction. Change this section to change what features we are using. 

  input: a directory of the data. Will split to test and train, a boolean: if split is true, will extract features and labels and store them in 4 arrays:
  train_data, train_label, test_data, test_label. if split is false, function will extract all data provided in the directory and output 2 arrays with values 2 arrays that are empty. 
  Use labeled data and split = false will give you  (train_data, train_label); use unlabled data and split = false will give you (test_data, test_label).

  output: data matrix and lables for both test and train. 

  '''
  audio_files = [f for f in os.listdir(dir) if f.endswith(".wav")]
  
  train_files, test_files = train_test_split(audio_files, test_size=0.2)

  train_data = []
  train_label= []
  test_data = []
  test_label = []  
  
  
  for file_name in audio_files:

      # Load audio
      file_path = os.path.join(dir, file_name)
      raw_audio, sr = librosa.load(file_path)

      # if audio is too short, append zeros after it. 
      if np.shape(raw_audio)[0] < 80000: 
          padded_audio = np.pad(raw_audio, [(0, 80000 - np.shape(raw_audio)[0])], mode='constant')
      else:
          padded_audio = raw_audio

      # now slice so that we guarantee that each array has the same length
      audio = padded_audio[20000:80000]

      # feature extraction

      mfccs = librosa.feature.mfcc(y=audio, sr = 22050, n_mfcc = 100)

      #chroma_stft = librosa.feature.chroma_stft(y = audio, sr=22050)
      #mel_spec = librosa.feature.melspectrogram(y = audio, sr = 22050, n_mels = 5)
      #gfccs = librosa.feature.gfcc(y, sr=sr, n_mfcc=20)

      # Concatenate
      #features = np.concatenate([mfccs.flatten()])
      features = mfccs

      label_map = {'angry': 0, 'calm': 1, 'disgust': 2, 'fearful': 3, 'happy': 4, 'neutral': 5, 'sad': 6, 'surprised': 7}
      label = file_name.split(".")[0]

      #unlabeled data
      if label[:-3] == "sample":
        test_data.append(features)
        test_label.append(label)

      #labeled data
      else:
        if split: 
          if file_name in train_files:
            train_data.append(features)
            label = re.sub('[^a-z]', "", label)
            train_label.append(label_map[label])

          elif file_name in test_files: 
            test_data.append(features)
            label = re.sub('[^a-z]', "", label)
            test_label.append(label_map[label])

        elif not split: 
          
          train_data.append(features)
          label = re.sub('[^a-z]', "", label)
          train_label.append(label_map[label])
  

  train_data = np.array(train_data)
  train_label = np.array(train_label)
  test_data = np.array(test_data)
  test_label = np.array(test_label) 

  print("mfccs size is:", np.shape(mfccs))
  return train_data, train_label, test_data, test_label



In [17]:
X_train, y_train, X_test, y_test = FeatureExtraction(data_dir, split = True)
y_train_ohe = to_categorical(y_train)
y_test_ohe = to_categorical(y_test)


mfccs size is: (100, 118)


In [ ]:
#@title SVM
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
clf = make_pipeline(StandardScaler(), SVC(kernel = "linear"))
clf.fit(X_train, y_train)

y_predicted= clf.predict(X_test)

accuracy = accuracy_score(y_test, y_predicted)

print(accuracy)

ValueError: ignored

Below is for unlabeled test data and importing a csv


In [8]:
clf = make_pipeline(StandardScaler(), SVC(kernel = "linear"))
clf.fit(X_train, y_train)

y_predicted= clf.predict(X_test)

NameError: ignored

In [9]:
import keras
from keras.layers import Conv1D, Embedding, Dropout, MaxPooling1D, GlobalMaxPooling1D, Dense
from keras.optimizers import Adam
from keras import Model
from keras.callbacks import EarlyStopping

In [10]:

num_classes = 8
def build_cnn(input_shape, num_classes):
    model = tf.keras.Sequential()
    
    # Convolutional layers
    model.add(tf.keras.layers.Conv2D(64, (3, 3), padding='valid', input_shape=input_shape))
    model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.Activation('relu'))
    model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))

    model.add(tf.keras.layers.Conv2D(128, (3, 3), padding='valid'))
    model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.Activation('relu'))
    model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))

    model.add(tf.keras.layers.Conv2D(256, (3, 3), padding='valid'))
    model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.Activation('relu'))
    model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))
    model.add(tf.keras.layers.MaxPooling2D())

    # Dense layers
    model.add(tf.keras.layers.Flatten())
    model.add(tf.keras.layers.Dense(128))
    model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.Activation('relu'))
    model.add(tf.keras.layers.Dropout(0.3))
    
    model.add(tf.keras.layers.Dense(num_classes))
    model.add(tf.keras.layers.Activation('softmax'))
    
    return model

In [ ]:
#num frames is number of wav files
es = EarlyStopping(monitor='val_loss',
                   mode='min',
                   restore_best_weights=True,
                   patience=10,
                   verbose=1)

#num frames is number of wav files
model = build_cnn((X_train.shape[1], X_train.shape[2], 1), num_classes)
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
history = model.fit(X_train, y_train_ohe, batch_size=32, epochs=15, validation_data=(X_test, y_test_ohe), callbacks=[es])

plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Training and validation error')
plt.xlabel('Epoch')
plt.ylabel('Error')
plt.legend(['Training', 'Validation'], loc='upper right')
plt.show()

Epoch 1/15
29/29 [==============================] - 7s 48ms/step - loss: 1.7827 - accuracy: 0.3856 - val_loss: 28.3192 - val_accuracy: 0.1067
Epoch 2/15
29/29 [==============================] - 1s 39ms/step - loss: 1.1389 - accuracy: 0.5922 - val_loss: 19.4067 - val_accuracy: 0.1067
Epoch 3/15
29/29 [==============================] - 1s 39ms/step - loss: 0.9313 - accuracy: 0.6744 - val_loss: 11.9827 - val_accuracy: 0.1067
Epoch 4/15
29/29 [==============================] - 1s 39ms/step - loss: 0.6368 - accuracy: 0.8067 - val_loss: 2.5265 - val_accuracy: 0.2133
Epoch 5/15
29/29 [==============================] - 1s 39ms/step - loss: 0.4937 - accuracy: 0.8544 - val_loss: 2.0768 - val_accuracy: 0.4044
Epoch 6/15
29/29 [==============================] - 1s 38ms/step - loss: 0.3341 - accuracy: 0.9178 - val_loss: 4.5738 - val_accuracy: 0.2489
Epoch 7/15
27/29 [==========================>...] - ETA: 0s - loss: 0.2684 - accuracy: 0.9456

In [12]:
# extract all labeled data
X_train, y_train, dummy1, dummy2 = FeatureExtraction(data_dir, split = False)
# extract all unlabeled data
dummy1, dummy2, X_test, y_test = FeatureExtraction(test_dir, split = False)
y_train_ohe = to_categorical(y_train)


mfccs size is: (100, 118)
mfccs size is: (100, 118)


In [13]:
# this cell is to export a .csv file to submit (filename & predicted label) 
model = build_cnn((X_train.shape[1], X_train.shape[2], 1), num_classes)
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


model.fit(X_train, y_train_ohe, batch_size=32, epochs=20)
y_predicted = model.predict(X_test)



Epoch 1/20
36/36 [==============================] - 5s 43ms/step - loss: 1.6785 - accuracy: 0.4436
Epoch 2/20
36/36 [==============================] - 1s 35ms/step - loss: 1.0022 - accuracy: 0.6631
Epoch 3/20
36/36 [==============================] - 1s 33ms/step - loss: 0.7456 - accuracy: 0.7529
Epoch 4/20
36/36 [==============================] - 1s 33ms/step - loss: 0.5674 - accuracy: 0.8249
Epoch 5/20
36/36 [==============================] - 1s 33ms/step - loss: 0.4019 - accuracy: 0.8960
Epoch 6/20
36/36 [==============================] - 1s 33ms/step - loss: 0.2679 - accuracy: 0.9413
Epoch 7/20
36/36 [==============================] - 1s 33ms/step - loss: 0.1393 - accuracy: 0.9796
Epoch 8/20
36/36 [==============================] - 1s 33ms/step - loss: 0.1510 - accuracy: 0.9724
Epoch 9/20
36/36 [==============================] - 1s 33ms/step - loss: 0.1055 - accuracy: 0.9831
Epoch 10/20
36/36 [==============================] - 1s 32ms/step - loss: 0.0843 - accuracy: 0.9911
Epoch 11/

In [15]:

y_predicted_ohv = np.argmax(y_predicted, axis = 1)


In [22]:
print(y_predicted_ohv)
label_map = {'angry': 0, 'calm': 1, 'disgust': 2, 'fearful': 3, 'happy': 4, 'neutral': 5, 'sad': 6, 'surprised': 7}

def dict_search(dictionary, value):
    for key, val in dictionary.items():
        if val == value:
            return key

y_predicted_label = []
for i in y_predicted_ohv: 
    val = dict_search(label_map, i)
    y_predicted_label.append(val)


print(y_predicted_label)

[1 3 6 1 2 4 1 3 4 2 3 2 2 2 2 6 2 2 3 2 7 6 4 6 3 1 1 1 7 0 7 4 3 1 0 2 3
 3 4 0 3 3 1 2 2 1 2 2 2 4 3 2 0 0 6 3 3 5 2 3 7 3 6 3 4 4 3 0 4 0 3 2 6 7
 1 7 3 2 3 4 6 7 2 7 3 4 6 2 3 2 3 7 2 4 1 7 2 2 2 3 2 2 2 7 7 3 6 1 3 4 6
 1 3 3 4 7 3 3 7 0 2 2 7 7 7 7 2 3 7 1 7 7 7 7 4 7 4 3 4 1 2 1 2 0 3 7 3 3
 7 4 1 7 6 0 3 2 3 3 7 3 7 4 1 7 1 1 1 4 2 2 3 3 3 0 4 0 7 3 2 1 2 3 7 1 2
 5 6 4 0 0 7 1 0 3 0 3 3 2 3 2 7 1 0 0 2 1 7 3 7 1 1 6 3 1 0 5 0 4 2 4 3 3
 4 3 6 1 1 2 3 2 1 3 3 3 3 2 0 2 2 5 3 7 4 1 0 1 3 7 0 3 7 4 4 2 7 0 6 4 3
 6 6 4 3 1 7 2 2 6 0 4 2 3 2 3 2 2 7 7 1 4 4 1 4 3 1 2 3 4 4 4 7 1 1 7 4 4
 1 2 1 2 1 7 4 3 3 2 4 2 4 0 1 3 3 1 2]
['calm', 'fearful', 'sad', 'calm', 'disgust', 'happy', 'calm', 'fearful', 'happy', 'disgust', 'fearful', 'disgust', 'disgust', 'disgust', 'disgust', 'sad', 'disgust', 'disgust', 'fearful', 'disgust', 'surprised', 'sad', 'happy', 'sad', 'fearful', 'calm', 'calm', 'calm', 'surprised', 'angry', 'surprised', 'happy', 'fearful', 'calm', 'angry', 'disgust', 'fearf

In [2]:
import csv

def make_csv(unlabeled_file_name, y_predicted_label):
  with open('prediction.csv', mode='w', newline='') as file:
      writer = csv.writer(file)
      writer.writerow(['filename', 'label'])
      for i in range(len(unlabeled_file_name)):
        writer.writerow([unlabeled_file_name[i], y_predicted_label[i]])
  file.close()

make_csv(y_test, y_predicted_label)

NameError: ignored